# Constraint Satisfaction Homework

## Code in hw2cs561s2018.py file

### Read in file

In [3]:
#import sys
#filePath = sys.argv[1]
with open('tests/input1.txt', 'r') as openFile:
  fileData = openFile.readlines()
openFile.close()
cspInfo = [line.strip() for line in fileData]

### Assign variables from values provided in value

In [4]:
import collections
dNumOfGroups = int(cspInfo[0])
dNumOfPots = int(cspInfo[1])
lsVariables = []
dctVarInfo = collections.defaultdict(list)
dctDomains = collections.defaultdict(list)
count = 2
intPotNum = 1
for pot in range(dNumOfPots):
    splitLine = cspInfo[count].split(',')
    for country in splitLine:
        lsVariables.append(country)
        dctVarInfo[country].append(intPotNum)

        for num in range(1, dNumOfGroups+1):
            dctDomains[country].append(num)

    count = count + 1
    intPotNum = intPotNum + 1
for confederation in range(6):
    #parse string
    splitLine = cspInfo[count].split(':')
    strConfederation = splitLine.pop(0)
    splitLine = str(splitLine)
    splitLine = splitLine.split(',')
    count = count + 1
    if 'None' not in ''.join(splitLine):
        for word in splitLine:
            word = word.strip('[')
            word = word.strip(']')
            word = word.strip("'")
            dctVarInfo[word].append(strConfederation)

In [6]:
# Create neighbors variable and assign neighbors related to values that 
# can be assigned. For example: If variable country is in pot 1; then its
# neighbors can be 2, 3, and/or 4.
import collections
dctNeighboringCountries = collections.defaultdict(list)
for country in dctVarInfo.keys():
    for checkCountry in lsVariables:
        if country != checkCountry:
            if dctVarInfo[country][0] == dctVarInfo[checkCountry][0]:
                if checkCountry not in dctNeighboringCountries[country]:
                    dctNeighboringCountries[country].append(checkCountry)
            if dctVarInfo[country][1] == dctVarInfo[checkCountry][1]:
                if checkCountry not in dctNeighboringCountries[country]:
                    dctNeighboringCountries[country].append(checkCountry)

### Send variables to CSP file to be solved and returned.

In [8]:
#Analyze size of the problem, and for large problems use hill climbing, for smaller problems use backward checking.
import collections
import CSP
import SearchAndCheck
dctAssignments = {}
if len(lsVariables) >= 20:
    consistencyCheck = 'hc'
else:
    consistencyCheck = 'bc'
cspProblem = CSP.CSP(lsVariables, dctVarInfo, dctDomains, dctNeighboringCountries, None)
solve = SearchAndCheck.SearchAndCheck(cspProblem)
if consistencyCheck == 'bc':
    solution = solve.BacktrackSearch(dctAssignments, consistencyCheck)
else:
    solution = solve.HillClimbing(1000, dctAssignments)

AttributeError: 'collections.defaultdict' object has no attribute 'iteritems'

## Code from CSP.py 

### Class declaration

In [9]:
import collections

class CSP(object):
    def __init__(self, variables, variableInfo, domains, potNeighbors, confNeighbors, constraints):
        self.variables = variables 
        self.valInfo = variableInfo
        self.domains = domains
        self.notConstrainedDomains = None  #Will be updated along the search process
        self.neighboringCountries = neighbors
        self.constraints = constraints
        self.assignOrder = collections.OrderedDict() #When assignment is made add to this dictionary, when deletion is needed remove most recently added
        self.dctConstraints = {} #Dictionary containing a country and the constraints on that country.
        self.assignments = collections.OrderedDict()
        self.consCheck = collections.defaultdict(list)

### Functions related to assignments during the problem solving processes.

In [10]:
#Remove assignments takes in the country that was unassignable and the dictionary of current assignments. It removed the country from assignemtns and return the update dictionary.
def RemoveAssignment(self, country, assignments):
    if country in assignments:
        del self.assignments[country]
    return self.assignments

#Add country and group to assignment dictionary and return.
def MakeAssignment(self, country, group, assignments):
    self.assignments[country] = group
    return self.assignments

#Take in a list of removed countries and the values(domains) pruned and re-add them to possible domains (notConstrainedDomains)
def Undo(self, lsRemoved):
    for removedCountry, removedValue in lsRemoved:
        self.notConstrainedDomains[removedCountry].append(removedValue)

### Functions related to checking constraints.

In [11]:
#IsConstrained: Check a direct constraint between two countries and their values.
    #Return True is the country:group combination is constrained; therefore, can't be made.
    #Return False if not constrained. 

def IsConstrained(self, country, cValue, neighbor, nValue):
    if ((cValue == nValue) and (self.valInfo[country][0] == self.valInfo[neighbor][0])):
      return True
    elif ((cValue == nValue) and ('UEFA' in self.valInfo[neighbor])):
        uefaCount = 0
        for country, value in self.assignments.iteritems():
            if ((value == cValue) and ('UEFA' in self.valInfo[country])):
                uefaCount = uefaCount + 1
        if uefaCount < 2:
            return False
        else:
            return True
    elif cValue == nValue:
        return True

    return False

#GetConstraintCount: Get the total count of constraints for each unassigned country for the current assignments made. Return the dictionary that contains the country and number of
# constraints.

def GetConstraintCount(self, variables, assignments):
    valInfo = copy.deepcopy(self.valInfo)
    dctConstraints = {}
    for country in variables:
        constraints = 0
        possVals = self.domains[country]
        if country not in assignments:
            for value in possVals:
                uefaCount = 0
                uefaFlip = False
                for checkCountry, checkValue in assignments.iteritems():
                    if value == checkValue and valInfo[country][0] == valInfo[checkCountry][0]:
                        constraints = constraints + 1
                    if value == checkValue and valInfo[country][1] == valInfo[checkCountry][1]:
                        if 'UEFA' in valInfo[checkCountry][1]:
                            uefaCount = uefaCount + 1
                        elif ('UEFA' not in valInfo[checkCountry][1]):
                            constraints = constraints + 1
                    if ((uefaCount == 2) and ('UEFA' in valInfo[country][1])):
                        if uefaFlip == False:
                            constraints = constraints + 1
                            uefaFlip = True
            dctConstraints[country] = constraints

    self.dctConstraints = dctConstraints
    return dctConstraints

#CountryConstraints: Get the total count of constraints for a specific country for the current assignments.

def CountryConstraints(self, country, assignments):
    valInfo = copy.deepcopy(self.valInfo)
    value = assignments[country]
    constraints = 0
    uefaCount = 0
    uefaFlip = False
    for checkCountry, checkValue in assignments.iteritems():
        if checkCountry != country:
            if value == checkValue and valInfo[country][0] == valInfo[checkCountry][0]:
                constraints = constraints + 1
            if value == checkValue and valInfo[country][1] == valInfo[checkCountry][1]:
                if 'UEFA' in valInfo[checkCountry][1]:
                    uefaCount = uefaCount + 1
                elif ('UEFA' not in valInfo[checkCountry][1]):
                    constraints = constraints + 1
            if ((uefaCount == 2) and ('UEFA' in valInfo[country][1])):
                if uefaFlip == False:
                    constraints = constraints + 1
                    uefaFlip = True
    return constraints

#CheckConstraint: Return true if a given country:group pair is valid within the current assignments. Return true if no constraints are found.


def CheckConstraints(self, currentCountry, value, assignments):
    constraints = 0
    uefaCount = 0
    for country, checkValue in assignments.iteritems():
        if value == checkValue:
            if self.valInfo[country][0] == self.valInfo[currentCountry][0]:
                return 1
            elif ((self.valInfo[country][1] == self.valInfo[currentCountry][1]) and ('UEFA' not in self.valInfo[country][1])):
                return 1
            elif ((self.valInfo[country][1] == self.valInfo[currentCountry][1]) and ('UEFA' in self.valInfo[country][1])):
                uefaCount = uefaCount + 1
            if uefaCount >= 2:
                return 1
    return constraints

#ConflictedAssignments: Used for hill climbing search to return all countries within a given assignment that are constrained.


def ConflictedAssignments(self, assignments):
    countries = []
    for country, group in assignments.iteritems():
        if self.CheckConstraints(country, group, assignments) != 0:
            countries.append(country)
    return countries

## Code from SearchAndCheck.py 

### CSP Algorithms: HillClimbing and Backtracking

In [12]:
#Hill climbing's randomization will be useful in large problems as backtracking seems to loop causing time issues.
def HillClimbing(self, steps, dctAssignments):
    lsVariables = self.csp.variables
    toChange = []

    for country in lsVariables:
        if self.csp.notConstrainedDomains:
            lsReorderedVals = self.OrderValues(self.csp.notConstrainedDomains[country], dctAssignments)
        else:
            lsReorderedVals = self.OrderValues(self.csp.domains[country], dctAssignments)
        if len(lsReorderedVals):
            dctAssignments[country] = lsReorderedVals.pop(0)

    #Once all variables have been assigned go through and find conflicts and change assignments
    for step in range(steps):
        conflict = False
        random.shuffle(lsVariables)
    for randomToChangeCountry in lsVariables:
        if self.csp.CountryConstraints(randomToChangeCountry, dctAssignments) == 0:
            continue
        minConstraintCount = self.csp.CountryConstraints(randomToChangeCountry, dctAssignments)
        minConstrainedGroups = []
        for group in self.csp.domains[randomToChangeCountry]:
            dctAssignments[randomToChangeCountry] = group
            constraintCount = self.csp.CountryConstraints(randomToChangeCountry, dctAssignments)
            if constraintCount == minConstraintCount:
                minConstrainedGroups.append(group)
            elif constraintCount < minConstraintCount:
                minConstraintCount = constraintCount
                del minConstrainedGroups[:]
                minConstrainedGroups.append(group)
        groupToChange = random.choice(minConstrainedGroups)
        dctAssignments[randomToChangeCountry] = groupToChange
        conflict = True
    if not conflict:
        return dctAssignments
    #If problem gets to this point no solution so return None
    return None


#Fast for smaller problems so use for those.
def BacktrackSearch(self, dctAssignments, consistencyCheck):
    lsVariables = self.csp.variables
    if len(dctAssignments) == len(self.csp.valInfo):
        return dctAssignments
    if len(dctAssignments) == 0:
        length = 0
        for country, neighbors in self.csp.neighboringCountries.iteritems():
            if len(neighbors) > length:
                length = len(neighbors)
                strCurrentCountry = country

    else:
        strCurrentCountry = self.GetVarMRV(lsVariables, dctAssignments)

    if len(dctAssignments) == 0:
        lsReorderedVals = self.csp.domains[strCurrentCountry]
    else:
        if self.csp.notConstrainedDomains:
            lsReorderedVals = self.OrderValues(self.csp.notConstrainedDomains[strCurrentCountry], dctAssignments)
        else:
            lsReorderedVals = self.OrderValues(self.csp.domains[strCurrentCountry], dctAssignments)

    for value in lsReorderedVals:
        if 0 == self.csp.CheckConstraints(strCurrentCountry, value, dctAssignments):
            dctAssignments = self.csp.MakeAssignment(strCurrentCountry, value, dctAssignments)
            # Begin Inference
            removed = self.MakeInferenceFromRemoval(strCurrentCountry, value)
            self.removed = removed
            consistent = self.consistent(strCurrentCountry, value, dctAssignments)
            if consistent:
                solution = self.BacktrackSearch(dctAssignments, consistencyCheck)
                if solution != None:
                    return solution
            self.csp.Undo(removed)

    self.csp.RemoveAssignment(strCurrentCountry, dctAssignments)
    return None


### Functions used by the search algorithms:

In [13]:
#CheckForPrune: Takes a country and one of its neighbors and checks if any values are constrained, if so it removes that group number from possible domain values.

def CheckForPrunes(self, neighbor, currentCountry):
    prunesAvailable = False
    for domainValue in self.csp.notConstrainedDomains[currentCountry]:
        if all(not self.csp.IsConstrained(currentCountry, domainValue, neighbor, value) for value in self.csp.notConstrainedDomains[neighbor]):
            self.MakePrune(neighbor, value)
            prunesAvailable = True
    return prunesAvailable

#Consistent: Takes a country and a group number and checks if those values are not constrained with that assignment for all the current assignments.

def consistent(self, country, value, assignments):
    for neighbor in self.csp.neighboringCountries[country]:
        if neighbor not in assignments:
            for possibleValue in self.csp.notConstrainedDomains[neighbor]:
                constrained = self.csp.IsConstrained(country, value, neighbor, possibleValue)
                if constrained:
                    self.MakePrune(neighbor, possibleValue)
        if neighbor in self.csp.notConstrainedDomains:
            if not self.csp.notConstrainedDomains[neighbor]:
                return False
    return True

#MakeInferenceFromRemoval: Takes an assignment and removes all other possible groups(domain values)

def MakeInferenceFromRemoval(self, country, valueAssigned):
    self.PruneSetup()
    removed = [(country, value) for value in self.csp.notConstrainedDomains[country] if value != valueAssigned]
    self.csp.notConstrainedDomains[country] = [valueAssigned]
    return removed

#MakePrune: Remove value from possible domains and add it removed list.

def MakePrune(self, country, value):
    self.csp.notConstrainedDomains[country].remove(value)
    if self.removed is not None:
        self.removed.append((country, value))
        
#PruneSetup: Assign notConstrainedDomains from CSP to allow for removal of impossible group values.


def PruneSetup(self):
    if not self.csp.notConstrainedDomains:
        self.csp.notConstrainedDomains = {variable: list(self.csp.domains[variable]) for variable in self.csp.variables}
        
#Order values: Take in possible domain values and current assignments order the values based on the values that cause the least constraints.
  
def OrderValues(self, domainInfo, dctAssignments):
    if len(domainInfo) == 1:
        return domainInfo
    dctValInfo = collections.defaultdict(list)
    lsOrderedValues = []
    for country, value in dctAssignments.iteritems():
        if value in domainInfo:
            dctValInfo[value].append(1)
    for value in sorted(dctValInfo, key=lambda value: len(dctValInfo[value]), reverse=True):
        lsOrderedValues.append(value)
    if len(lsOrderedValues) == 0 and len(domainInfo) != 0:
        lsOrderedValues = domainInfo
    return lsOrderedValues

#GetVarMRV: Get next variable to assign based MRV

def GetVarMRV(self, variables, dctAssignments):
    dctCount = []
    for var in variables:
        if var not in dctAssignments:
            variable = var
            break

    mrv = variable
    dctConstraintCount = self.csp.GetConstraintCount(variables, dctAssignments)
    dctCount.append(variable)
    for country, count in dctConstraintCount.iteritems():
        if country != variable:
            if dctConstraintCount[country] > dctConstraintCount[variable]:
                variable = country
                dctCount.pop(0)
                dctCount.append(country)
            elif dctConstraintCount[country] == dctConstraintCount[variable]:
                dctCount.append(country)
    if len(dctCount) == 1:
        mrv = dctCount[0]
    elif len(dctCount) > 1:
        mrv = dctCount.pop(0)
        if mrv in self.csp.notConstrainedDomains:
            mrvCount = len(self.csp.notConstrainedDomains[mrv])
        else:
            mrvCount = len(self.csp.domains[mrv])
        for checkCountry in dctCount:
            if checkCountry in self.csp.notConstrainedDomains:
                cCount = len(self.csp.notConstrainedDomains[checkCountry])
            else:
                cCount = len(self.csp.domains[checkCountry])
            if cCount <= mrvCount:
                mrvCount = cCount
                mrv = checkCountry
    return mrv